Step 1: Getting the data into Python, and cleaning it.
- will need to write code to import and clean, then functionalize it.

Steps to clean data:

*Remaining balance:
- need to remove $ and ,
- convert to integer
pandas already did it

*location:
city names contain misspellings and characters
state names are not all abbreviated
some zip codes have postal codes
- start by assigning all blank values as missing  
- pull only the first part of the zip into a new col
- if original zip col is not missing look zip up in zippopotamus to return city and state, 
- if original zip is missing or blank, look up city and state
- if city and state are missing, return missing

Language:
-correct language blanks to missing

*DOB:
-assign DOBs before today as NA

Marital status:
- assign blanks to missing

*Gender:
- assign blanks to missing

*Race:
- some values for white misspelled
- some values for American Indian misspelled
- if contains american indian, then American Indian or Alaska Native
 - if starts with W, then white
- assign blanks to missing

*Hispanic/Latino:
- some values mispelled
- non-hispanic or latino not consistent
- some values no
- assign blanks to missing
- Everything that starts with no should be assigned to non-hispanic or latino
- if doesn't start with no or is missing, decline to answer,  or non-hispanic, assign to Hispanic or Latino

*Sexual orientation:
- assign blanks and N/As to missing
- assign decline to "decline to answer"
- If it starts with st assign to straight

*Insurance type:
- if contains medicare or medicaid, assign to Medicare & Medicaid
- if starts with un then uninsured
- if missing or blank, assign missing

Household size:
- assign the row with 4602 to blank
- assign missings to blank?

*Household income:
- remove $ - and , assign as integers
- assign missings to blank
pandas already did it

*Distance round trip:
- take only numbers, assign text to missing

referral source:
- assign blanks to missing

*Amount:
 - take only numbers, remove $ - and ,
- assign blanks to missing

*payment method:
only text, assign responses with only numbers to missing

payable to:
Surely I don't have to do anything with this

*patient letter notified:
- assign na, n/a, missing, and blanks to No
- assign dates to Y

Application signed:
should be fine.

other:
make sure data types align with what's needed
- use type "object" to handle numerical and non-numerical data?
Bulleted cols need to be cleaned



To-do:
-cleaning for address using zipopotamus api
-lesson 9 to guide

In [ ]:
import pandas as pd
import numpy as np
import requests
import os
import sys
import re

#function to fetch zip codes
def fetch_zip_info(zip_codes):
    """Fetch city and state info for zip codes."""
    zip_to_locale = {}
    for zip_code in zip_codes:
        try:
            url = f"https://api.zippopotam.us/us/{zip_code}"
            response = requests.get(url)
            if response.status_code == 200:
                zip_data = response.json()
                city = zip_data['places'][0]['place name']
                state = zip_data['places'][0]['state abbreviation']
                zip_to_locale[zip_code] = {'City': city, 'State': state}
            else:
                zip_to_locale[zip_code] = {'City': 'Unknown', 'State': 'Unknown'}
        except Exception:
            zip_to_locale[zip_code] = {'City': 'Error', 'State': 'Error'}
    return zip_to_locale

def clean_data(filepath, sheet_name="None"):
    """Clean the service learning data."""
    today = pd.Timestamp.today()

    # Read file (Excel or CSV)
    if filepath.endswith('.xlsx'):
        data = pd.read_excel(filepath, sheet_name=sheet_name)
    elif filepath.endswith('.csv'):
        data = pd.read_csv(filepath)
    else:
        raise ValueError("Unsupported file format: Only .csv or .xlsx allowed.")

    # Clean Payment Submitted Date
    def extract_date(text):
        if isinstance(text, pd.Timestamp):
            return text.strftime("%m/%d/%Y")
        match = re.search(r'\b(\d{1,2}[-/]\d{1,2}[-/]\d{2,4})\b', str(text))
        return match.group(0) if match else None
    # Extract dates
    data['Payment Submitted Date'] = data['Payment Submitted?'].apply(extract_date)
    # Convert to datetime
    data['Payment Submitted Date'] = pd.to_datetime(data['Payment Submitted Date'], errors='coerce')
    # Ensure the column is explicitly set to 'Yes' if a valid date was found
    has_date = data['Payment Submitted Date'].notna()
    data.loc[has_date, 'Payment Submitted?'] = 'Yes'

    # Clean Zip, City, State
    data['Pt Zip'] = data['Pt Zip'].astype(str).str.strip().str.extract(r'(\d{5})')[0]
    data['Pt Zip'] = data['Pt Zip'].fillna("Missing")

    data.loc[data['Pt Zip'] == "Missing", ['Pt City', 'Pt State']] = "Missing"

    valid_zips = data[data['Pt Zip'] != "Missing"]['Pt Zip'].unique()
    zip_to_locale = fetch_zip_info(valid_zips)

    data['Pt City'] = data['Pt Zip'].apply(lambda z: zip_to_locale.get(z, {}).get('City', 'Missing'))
    data['Pt State'] = data['Pt Zip'].apply(lambda z: zip_to_locale.get(z, {}).get('State', 'Missing'))

    # Clean DOB
    data['DOB'] = pd.to_datetime(data['DOB'], errors='coerce')
    data.loc[data['DOB'] > today, 'DOB'] = pd.NaT
 
    # Clean Gender
    data['Gender'] = data['Gender'].replace(r'^\s*$', "Missing", regex=True)

    # Clean Race
    data['Race'] = data['Race'].astype(str).str.strip().str.lower()
    data['Race'] = data['Race'].apply(lambda x: (
        'American Indian or Alaska Native' if 'american indian' in x else
        'White' if x.startswith('w') else
        "Missing" if x in ['', 'nan'] else x.title()
    ))

    # Clean Hispanic/Latino
    data['Hispanic/Latino'] = data['Hispanic/Latino'].astype(str).str.strip().str.lower()
    data['Hispanic/Latino'] = data['Hispanic/Latino'].apply(lambda x: (
        'Non-Hispanic or Latino' if x.startswith('no') else
        'Hispanic or Latino' if not (x.startswith('no') or x in ['nan', '', 'missing', 'decline to answer', 'non-hispanic']) else
        np.nan
    ))

    # Clean Sexual Orientation
    data['Sexual Orientation'] = data['Sexual Orientation'].astype(str).str.strip().str.lower()
    data['Sexual Orientation'] = data['Sexual Orientation'].apply(lambda x: (
        'Decline to answer' if x == 'decline' else
        'Straight' if x.startswith('st') else
        np.nan if x in ['n/a', '', 'nan'] else x.title()
    ))

    # Clean Insurance Type
    data['Insurance Type'] = data['Insurance Type'].astype(str).str.strip().str.lower()
    data['Insurance Type'] = data['Insurance Type'].apply(lambda x: (
        'Medicare & Medicaid' if 'medicare' in x or 'medicaid' in x else
        'Uninsured' if x.startswith('un') else
        'Missing' if x in ['', 'nan'] else
        x.title()
    ))

    # Marital Status, Gender, Hispanic/Latino, Sexual Orientation blanks to "Missing"
    for col in ['Marital Status', 'Gender', 'Hispanic/Latino', 'Sexual Orientation']:
        data[col] = data[col].astype(str).str.strip().replace(r'^\s*$', 'Missing', regex=True).replace('nan', 'Missing')

    # Sexual Orientation further normalization
    data['Sexual Orientation'] = data['Sexual Orientation'].str.lower().apply(lambda x: (
        'Decline to answer' if x == 'decline' else
        'Straight' if x.startswith('st') else
        x.title()
    ))

    # Clean HouseHold Size
    data['Household Size'] = pd.to_numeric(data['Household Size'], errors='coerce')
    data.loc[(data['Household Size'] > 20) | (data['Household Size'].isna()), 'Household Size'] = np.nan

    # Clean Total Household Gross Monthly Income
    data['Total Household Gross Monthly Income'] = (
        data['Total Household Gross Monthly Income']
        .astype(str).str.replace(r'[^\d.]', '', regex=True)
    )
    data['Total Household Gross Monthly Income'] = pd.to_numeric(data['Total Household Gross Monthly Income'], errors='coerce')

    # Clean Distance roundtrip
    data['Distance roundtrip/Tx'] = pd.to_numeric(
        data['Distance roundtrip/Tx'].astype(str).str.extract(r'(\d+\.?\d*)')[0],
        errors='coerce'
    )

    # Clean Referral Source
    data['Referral Source'] = data['Referral Source'].astype(str).str.strip().replace(r'^\s*$', 'Missing', regex=True)

    # Clean Payment Method
    data['Payment Method'] = data['Payment Method'].astype(str).str.strip().replace(r'^\s*$', 'Missing', regex=True).replace('nan', 'Missing')
     # Strip everything except letters
    data['Payment Method'] = data['Payment Method'].str.replace(r'[^a-zA-Z\s]','', regex=True).str.strip()
    #Uppercase
    data['Payment Method'] = data['Payment Method'].str.upper()

    # Clean Remaining Balance
    data['Remaining Balance'] = pd.to_numeric(data['Remaining Balance'], errors='coerce').round(2)

    # Clean Patient Letter Notified
    def letter_notified(val):
        val = str(val).strip().lower()
        if val in ['na', 'n/a', 'missing', '', 'nan']:
            return 'No'
        try:
            pd.to_datetime(val)
            return 'Yes'
        except:
            return 'No'
    
    data['Patient Letter Notified? (Directly/Indirectly through rep)'] = data['Patient Letter Notified? (Directly/Indirectly through rep)'].apply(letter_notified)
       
    #Clean Payable to:
    # Replace blanks or whitespace-only strings with "Missing"
    data['Payable to:'] = data['Payable to:'].replace(r'^\s*$', "Missing", regex=True)

    # Export cleaned data for testing
    #output_path = r"C:\Users\Glen\Documents\Tools For Data Analysis\Semester Project\cleaned_data.csv"
    #data.to_csv(output_path, index=False)

    #Export cleaned data as part of github action
    output_path = os.path.join("output", "cleaned_data.csv")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    data.to_csv(output_path, index=False)
    
    return data

In [ ]:
#Testing function
#clean_data("C:\\Users\\Glen\\Documents\\Tools For Data Analysis\\Semester Project\\UNO Service Learning Data Sheet De-Identified Version.xlsx")

<positron-console-cell-18>:66: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.


,Patient ID#,Grant Req Date,App Year,Remaining Balance,Request Status,Payment Submitted?,Reason - Pending/No,Pt City,Pt State,Pt Zip,Language,DOB,Marital Status,Gender,Race,Hispanic/Latino,Sexual Orientation,Insurance Type,Household Size,Total Household Gross Monthly Income,Distance roundtrip/Tx,Referral Source,Referred By:,Type of Assistance (CLASS),Amount,Payment Method,Payable to:,Patient Letter Notified? (Directly/Indirectly through rep),Application Signed?,Notes,Payment Submitted Date
0,180001,2018-10-17,1,1180.00,Approved,Yes,NaN,Missing,Missing,Missing,Missing,NaT,Missing,Missing,Missing,Missing,Missing,Missing,NaN,NaN,NaN,NCS,Dr. Natarajan/Lily Salinas,Medical Supplies/Prescription Co-pay(s),320,MISSING,Missing,No,Missing,NaN,NaT
1,190001,2019-01-03,1,1428.39,Approved,Yes,NaN,Missing,Missing,Missing,Missing,NaT,Missing,Missing,Missing,Missing,Missing,Missing,NaN,NaN,NaN,NCS,Pam Owen/Sheri Shannon\n,Medical Supplies/Prescription Co-pay(s),21.61,MISSING,Missing,No,Missing,NaN,NaT
2,190001,2019-03-11,1,1428.39,Approved,Yes,NaN,Missing,Missing,Missing,Missing,NaT,Missing,Missing,Missing,Missing,Missing,Missing,NaN,NaN,NaN,NCS,Teresa Pfister,Food/Groceries,50,GC,Missing,No,Missing,NaN,NaT
3,190002,2019-05-20,1,1400.00,Approved,Yes,NaN,Missing,Missing,Missing,Missing,NaT,Missing,Missing,Missing,Missing,Missing,Missing,NaN,NaN,NaN,NCS,AG/Susan Keith,Food/Groceries,100,GC,Missing,No,Missing,NaN,NaT
4,190003,2019-05-22,1,1425.00,Approved,Yes,NaN,Missing,Missing,Missing,Missing,NaT,Missing,Missing,Missing,Missing,Missing,Missing,NaN,NaN,NaN,NCS,AG/Kristi McHugh,Medical Supplies/Prescription Co-pay(s),75,CC,Missing,No,Missing,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,240393,2025-01-31,2,1000.00,Pending,No,HS,Falls City,NE,68355,English,1960-09-23,Widowed,Female,Asian,Non-Hispanic or Latino,Straight,Uninsured,1.0,4000.0,100.0,CPN,NaN,Gas,500,MISSING,NaN,No,NaN,Waiting on HS,NaT
2288,240393,2025-01-31,2,1000.00,Pending,No,HS,Falls City,NE,68355,English,1960-09-23,Widowed,Female,Asian,Non-Hispanic or Latino,Straight,Uninsured,1.0,4000.0,100.0,CPN,NaN,Food/Groceries,500,MISSING,NaN,No,NaN,Waiting on HS,NaT
2289,240548,2025-01-31,2,1000.00,Pending,No,NaN,Fremont,NE,68025,English,1962-04-03,Married,Male,White,Non-Hispanic or Latino,Straight,Private,2.0,2895.0,15.0,NCS,ALISA SEIDLER,Multiple,1068.56,MISSING,NaN,No,NaN,NaN,NaT
2290,250038,2025-01-31,1,1500.00,Pending,No,NaN,Hastings,NE,68901,Spanish,1980-10-02,Single,Female,Other,Hispanic or Latino,Straight,Uninsured,2.0,918.0,2.0,Morrison Cancer Center,Kellie Sterkel-SW,Housing,1500,MISSING,NaN,No,NaN,NaN,NaT


In [ ]:
def main():
    if len(sys.argv) < 2:
        raise ValueError("❌ No input file provided. Usage: python clean_data_script.py <input_file>")

    input_file = sys.argv[1]
    output_file = os.path.splitext(input_file)[0] + "_CLEANED.csv"
    sheet_name = "PA Log Sheet" if input_file.endswith(".xlsx") else None

    cleaned_df = clean_data(input_file, sheet_name=sheet_name)
    cleaned_df.to_csv(output_file, index=False)
    print(f"✅ Cleaned {input_file} -> {output_file}")

if __name__ == "__main__":
    main()

GitHub action steps

name: Clean New Data

on:
  push:
    paths:
      - '**/*.csv'
      - '**/*.xlsx'
    branches:
      - main

jobs:
  clean-data:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout code
      uses: actions/checkout@v4

    - name: Set up Python
      uses: actions/setup-python@v5
      with:
        python-version: '3.x'

    - name: Install dependencies
      run: |
        pip install pandas numpy openpyxl

    - name: Detect changed file
      id: detect_file
      run: |
        CHANGED_FILE=$(git diff --name-only ${{ github.event.before }} ${{ github.sha }} | grep -E '\.csv$|\.xlsx$' | head -n 1)
        echo "CHANGED_FILE=$CHANGED_FILE" >> $GITHUB_ENV

    - name: Run data cleaning script
      if: env.CHANGED_FILE != ''
      run: |
        python clean_data_script.py "${{ env.CHANGED_FILE }}"

    - name: Commit cleaned data
      if: env.CHANGED_FILE != ''
      run: |
        git config --global user.name 'github-actions[bot]'
        git config --global user.email 'github-actions[bot]@users.noreply.github.com'
        git add *_CLEANED.csv
        git commit -m "Automated: Cleaned ${{ env.CHANGED_FILE }}" || echo "No changes to commit"
        git push


Creating Dashboard and Visualizations

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(
    page_title = "Pending Applications"
    layout = "wide"
    initial sidebar_state = 'expanded'
)

db_data = pd.read_csv("cleaned_data.csv")

